In [7]:
# Import Libraries Import Spacy
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

In [16]:
# Load Txt File
with open('key_events_20th_century.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [17]:
# Text Wrangling & Cleaning
import re

# Remove reference tags like [1], [12], etc.
text = re.sub(r'\[\d+\]', '', text)

# Normalize whitespace
text = re.sub(r'\s+', ' ', text)

# Replace known country name mismatches (update based on your list)
replacements = {
    'United States of America': 'United States',
    'USSR': 'Soviet Union',
    'U.S.': 'United States',
    'UK': 'United Kingdom',
    'Czechoslovakia': 'Czech Republic',  # optional, based on your country list
}
for old, new in replacements.items():
    text = text.replace(old, new)

# Save cleaned version (optional, for re-use)
with open('cleaned_key_events_20th_century.txt', 'w', encoding='utf-8') as f:
    f.write(text)

In [10]:
# Create NER Object
doc = nlp(text)

In [20]:
# Split Sentence Entities
sentences = list(doc.sents)
sentence_entities = []

for sent in sentences:
    ents = [ent.text for ent in sent.ents if ent.label_ == "GPE"]
    sentence_entities.append((sent.text, ents))

In [14]:
# Load your countries CSV
countries_df = pd.read_csv('countries_list_20th_century_1.5.csv')
countries = countries_df['country_name'].str.strip().tolist()

In [15]:
# Create filtered relationships
relationships = []

for sentence, ents in sentence_entities:
    involved = [ent for ent in ents if ent in countries]
    if len(involved) > 1:
        for i in range(len(involved)):
            for j in range(i+1, len(involved)):
                relationships.append((involved[i], involved[j], sentence))

In [18]:
# Cretae Relationships DataFrame
relationships_df = pd.DataFrame(relationships, columns=['Country1', 'Country2', 'Sentence'])
relationships_df.head()

,Country1,Country2,Sentence
0,France,Austria,After a period of diplomatic and military esca...
1,France,Hungary,After a period of diplomatic and military esca...
2,Austria,Hungary,After a period of diplomatic and military esca...
3,Germany,Russia,The Bolsheviks negotiated the Treaty of Brest-...
4,Germany,Italy,"Germany, 1933 Fascism first appeared in Italy ..."


In [19]:
# Save & Export
relationships_df.to_csv('country_relationships_20th.csv', index=False)